In [ ]:
# 计算图（实质是有向无环图DAG）
# 1.将代码分解成操作子
# 2.将计算表示成一个无环图：前向传播执行图并存储中间结果（求复合函数值），反向传播执行图并计算梯度、去除不需要的枝（求偏导数和梯度）
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool  # monkey-patch for deprecated np.bool usage

# 显式构造计算图（Tensorflow、Theano、MXNet）
from mxnet import sym
a=sym.var('a')  # 变量
b=sym.var('b')  # 变量
c=2*a+b  # 操作子（计算图的节点）
# 隐式构造计算图（PyTorch、MXNet）
from mxnet import autograd,nd
with autograd.record():
    a=nd.ones((2,1))
    b=nd.ones((2,1))
    c=2*a+b
# 微分求导链式法则；正向传播即计算图的前向遍历（从最右微分求导到最左微分求导），反向传播即计算图的后向遍历（从最左微分求导到最右微分求导）
# 平时高数练习题用的就是反向传播

In [17]:
# 自动求导
import torch
x=torch.arange(4.0)
print(x)

# 计算y关于x的梯度之前，需要一个地方来存储梯度
x.requires_grad_(True)  # 设置x为需要梯度的变量，等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # x的梯度，初始值为None；y关于x的梯度即存放于此
y=2*torch.dot(x,x)  # y是x的函数，即y=2*x1^2+2*x2^2+2*x3^2+2*x4^2
print(y) # y=2*14=28，因为之前设置x为需要梯度的变量，所以隐式构造计算图，有关梯度的函数存放于grad_fn
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()    # y对每个分量的梯度dy/dx_i存放于x.grad中
print(x.grad)  # 输出y对每个分量的梯度dy/dx_i，即dy/dxi

# 默认情况下，PyTorch会累加梯度到x.grad中，因此每次调用y.backward()前需清除旧值
x.grad.zero_()  # 清除梯度
y=x.sum()  # y是x的函数，即y=x1+x2+x3+x4
print(y)  # y=6，因为之前设置x为需要梯度的变量，所以隐式构造计算图，有关梯度的函数存放于grad_fn
y.backward()  # 计算y关于x的梯度
print(x.grad)  # 输出y对每个分量的梯度dy/dx_i，即dy/dxi

tensor([0., 1., 2., 3.])
tensor(28., grad_fn=<MulBackward0>)
tensor([ 0.,  4.,  8., 12.])
tensor(6., grad_fn=<SumBackward0>)
tensor([1., 1., 1., 1.])


In [21]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()  # 清除梯度
y=x*x  # y是x的函数，即y=xTx
u=y.detach()  # u是x的函数，但不需要梯度
z=u*x  # z是x的函数，即z=x1^3+x2^3+x3^3+x4^3
z.sum().backward()  # 计算z关于x的梯度，实际z关于x的梯度是u
x.grad==u  # 输出True，说明u的梯度没有被计算出来

# 不带参数的tensor.backward()假设tensor是一个标量
x.grad.zero_()  # 清除梯度
y.sum().backward()  # 计算y关于x的梯度
x.grad==2*x  # 输出True，说明y关于x的梯度是2*x

tensor([True, True, True, True])

In [ ]:
# 即使构建函数的计算图需要经过Python控制流（如循环、条件分支、函数调用等），PyTorch仍然可以计算得到变量的梯度
def f(a):
    b=a*2
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c
a=torch.randn(size=(),requires_grad=True)
d=f(a)
d.backward()
a.grad==d/a

tensor(True)